In [6]:
import datetime

"""
Gathering weather data
"""
import requests
import pandas as pd

# NYC coordinates
lat, lon = 40.7128, -74.0060

# Adjust start/end date dynamically if needed
# Get unique date from delay data
delay_date = (datetime.date.today() - datetime.timedelta(days=2)).isoformat()
start_date = delay_date
end_date = delay_date  # Same day, hourly granularity

# Open-Meteo API with temperature in Fahrenheit
weather_url = (
    f"https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={lat}&longitude={lon}"
    f"&start_date={start_date}&end_date={end_date}"
    f"&hourly=temperature_2m,precipitation,snowfall,relative_humidity_2m,windspeed_10m"
    f"&temperature_unit=fahrenheit"
    f"&timezone=America/New_York"
)

# Fetch and convert to DataFrame
response = requests.get(weather_url)
weather_data = response.json()
weather_df = pd.DataFrame(weather_data['hourly'])
weather_df['time'] = pd.to_datetime(weather_df['time'])

weather_df

,time,temperature_2m,precipitation,snowfall,relative_humidity_2m,windspeed_10m
0,2025-07-20 00:00:00,74.0,0.0,0.0,86,5.4
1,2025-07-20 01:00:00,74.0,0.0,0.0,87,5.6
2,2025-07-20 02:00:00,74.6,0.0,0.0,87,4.9
3,2025-07-20 03:00:00,74.4,0.1,0.0,90,6.0
4,2025-07-20 04:00:00,74.2,0.0,0.0,92,3.1
5,2025-07-20 05:00:00,73.8,0.0,0.0,94,1.1
6,2025-07-20 06:00:00,73.9,0.1,0.0,94,2.0
7,2025-07-20 07:00:00,74.4,0.7,0.0,90,2.3
8,2025-07-20 08:00:00,72.0,2.1,0.0,96,7.0
9,2025-07-20 09:00:00,78.1,0.0,0.0,80,5.1


In [2]:
import datetime
import pandas as pd
from sqlalchemy import create_engine, text

# === Database config ===
DB_USER = "postgres"
DB_PASSWORD = "commiteveryday"
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "train_delays"

# === Create DB engine ===
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

create_weather_hourly = """
CREATE TABLE IF NOT EXISTS weather_hourly (
    id SERIAL PRIMARY KEY,
    time TIMESTAMP,
    temperature_f REAL,
    precipitation REAL,
    snowfall REAL,
    humidity REAL,
    windspeed REAL
);
"""

with engine.connect() as conn:
    conn.execute(text(create_weather_hourly))
    print("Table created.")
    
# === Insert weather data ===
if not weather_df.empty:
    weather_cols = ['time', 'temperature_2m', 'precipitation', 'snowfall', 'relative_humidity_2m', 'windspeed_10m']
    
    df_weather = weather_df[weather_cols].copy()
    df_weather.columns = ['time', 'temperature_f', 'precipitation', 'snowfall', 'humidity', 'windspeed']
    
    df_weather.to_sql('weather_hourly', engine, if_exists='append', index=False)
    print(f"Inserted {len(df_weather)} weather records.")
else:
    print("No weather data found.")


Table created.
Inserted 24 weather records.


In [ ]:
nano ~/run_weather_script.sh

# If you convert to .py first:
*/15 7-8 * * 1-5 /usr/bin/python3 /Users/mitchel/Desktop/beep/mtaDelayPredictor/collect_real_time_mta_info.py

# Or to execute the notebook:
*/15 7-8 * * 1-5 /usr/local/bin/jupyter nbconvert --to notebook --execute /Users/mitchel/Desktop/beep/mtaDelayPredictor/collect_real_time_mta_info.ipynb